In [29]:
import os
os.getpid()

32017

In [31]:
import random
list2 = []
file_data = open('amazon_review_less_than_300_unbalanced.csv')
for row in file_data:
    list2.append(row)## Data pre-processing module
    
list3 = []
for i in range(len(list2)):
    list3.append((list2[i][-2],list2[i][:-3]))
    
random.seed(10)
random.shuffle(list3)

lenth = len(list3)
train_list = list3[0:int(lenth*0.8)]
test_list = list3[int(lenth*0.8):]




In [32]:

import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    # Modification: l represent train_list or test_list
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):

        self.corpora = corpora
        self.tokenizer = tokenizer

        d =  tokenizer.batch_encode_plus(corpora.sentences)
        
        padded_encoded_inputs = tokenizer.pad(d)
        
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [33]:
print(train_list[3000])

('2', 'this product made my soon break out worse:( fast delivery though.. just not right for my skin. good luck with yours:)')


In [34]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [35]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(3)

BATCH_SIZE = 8
num_classes = 5
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 5
CLIP = 1

In [36]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 400000/400000 [00:00<00:00, 1801381.00it/s]


creating training dataset ... 
creating training iterator ... 
creating test corpora ... 


100%|██████████| 100000/100000 [00:00<00:00, 2019706.36it/s]


creating test dataset ... 
creating test iterator ... 


In [39]:
list(training_iterator)[0][0].shape

torch.Size([8, 169])

In [40]:
classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)
classifier.load_state_dict(torch.load("results/best_model_BERT.pt"))



<All keys matched successfully>

In [41]:
from torch import optim
criterion = nn.CrossEntropyLoss(reduction = 'sum')

from tqdm import tqdm

In [55]:
errors = []
confusion_matrix = []

def evaluate(model, iterator, criterion):

    model.eval()
    confusion_matrix.append(torch.zeros(5,5))
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]-1).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) 
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[0][row][col] += 1
            if abs(row-col) >= 3:
                errors.append((ids[i], row, col))

            
 

In [56]:
print(list(dev_iterator)[0][0].shape)
print(list(dev_iterator)[0][1].shape)

torch.Size([8, 162])
torch.Size([8, 162])


In [57]:
evaluate(classifier, dev_iterator, criterion)   

12500it [06:31, 31.89it/s]


In [58]:
distance = 0
diagnal = 0

for i in range(5):
    for j in range(5):
        distance+= confusion_matrix[0][i][j]*abs(i-j)
        if i == j:
            diagnal += confusion_matrix[0][i][j]
        
print("dist:" ,distance)
print("diag: ",diagnal)

dist: tensor(41555.)
diag:  tensor(66204.)


In [60]:
confusion_matrix

[tensor([[ 5114.,   978.,   281.,   137.,   191.],
         [ 1296.,  2038.,   811.,   199.,   144.],
         [  416.,  1021.,  3923.,  1456.,   406.],
         [  153.,   351.,  1398.,  8734.,  4935.],
         [  494.,   987.,  1209., 16933., 46395.]])]

In [66]:
print(tokenizer.decode(errors[0][0]))
print("label: ", errors[0][1])
print("prediction", errors[0][2])

[CLS] this uv lamp makes it easy to have gel nails at home. choice of three time settings is a definite plus. super easy to use. works well. great merchant to work with. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(1, device='cuda:3')
prediction tensor(4, device='cuda:3')


In [67]:
print(tokenizer.decode(errors[10][0]))
print("label: ", errors[10][1])
print("prediction", errors[10][2])

[CLS] this same product at the store is at least a couple of dollars more expensive. and if you get the regular ( non baby ) aveeno you will pay even more money... save some money and get this one! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [68]:
print(tokenizer.decode(errors[100][0]))
print("label: ", errors[100][1])
print("prediction", errors[100][2])

[CLS] " when i received it, it was opened so their was no way of me knowing if something i was going to put on my face was safe. hate hate hate " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [69]:
print(tokenizer.decode(errors[1000][0]))
print("label: ", errors[1000][1])
print("prediction", errors[1000][2])

[CLS] " it is wonderful to find a question in a book. also wonderful to question a question. could this book ever be published in an environment of " " the circle " "? " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [70]:
print(tokenizer.decode(errors[2000][0]))
print("label: ", errors[2000][1])
print("prediction", errors[2000][2])

[CLS] " a little drop can make thin hair full and piecey looking. define and texturize. why they discontinued this product baffles me, i couldn't keep it in stock. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [71]:
print(tokenizer.decode(errors[2001][0]))
print("label: ", errors[2001][1])
print("prediction", errors[2001][2])

[CLS] " buying it from amazon you don't have to daal with wen with monthly withdrawals from your checking account. and the. errors which are made taking to much out of your account and getting it back. the product is excellent with routine use, you will notice the difference. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [72]:
print(tokenizer.decode(errors[2005][0]))
print("label: ", errors[2005][1])
print("prediction", errors[2005][2])

[CLS] " very cheerful when i smell this rosebud salve. always keep my lips moist, glossy and silky. everyday i take it with me. love it so much. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(1, device='cuda:3')
prediction tensor(4, device='cuda:3')


In [73]:
print(tokenizer.decode(errors[2105][0]))
print("label: ", errors[2105][1])
print("prediction", errors[2105][2])

[CLS] transformative. took a workshop with a foundation that used this book as the basis and wen to amazon to buy. read this [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(0, device='cuda:3')
prediction tensor(3, dev

In [75]:
print(tokenizer.decode(errors[1405][0]))
print("label: ", errors[1405][1])
print("prediction", errors[1405][2])

[CLS] " this book differs from the china plan you can eat fish, chicken and eggs. rest of dairy is out. it is also baded on protein. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [76]:
print(tokenizer.decode(errors[1305][0]))
print("label: ", errors[1305][1])
print("prediction", errors[1305][2])

[CLS] this is an excellent product. it performs exactly as advertised. it was easy to install and is easy to use. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(1, de

In [77]:
print(tokenizer.decode(errors[1306][0]))
print("label: ", errors[1306][1])
print("prediction", errors[1306][2])

[CLS] i have no problems with them so far. they arent overly soft but they are much softer than i had expected for such a low price. they came in on time and are packaged nicely. definitely would recommend them [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(1, device='cuda:3')


In [78]:
print(tokenizer.decode(errors[1307][0]))
print("label: ", errors[1307][1])
print("prediction", errors[1307][2])

[CLS] " perfect!!! on time, el producto es el misml de la foto, servicio enviado a tiempo, good!, nice, no more words ok " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(1, device='cuda:3')


In [79]:
print(tokenizer.decode(errors[1308][0]))
print("label: ", errors[1308][1])
print("prediction", errors[1308][2])

[CLS] this product is cheaply made. once applied to the skin it stops massaging with any pressure. i dropped it and it shattered into pieces after less than a week. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(3, device='cuda:3')
prediction tensor(0, device='cuda:3')


In [80]:
print(tokenizer.decode(errors[1312][0]))
print("label: ", errors[1312][1])
print("prediction", errors[1312][2])

[CLS] " i tried this one to save some money, but the rubis slant classic tweezer is better. the quality and build of the handle are pretty close, but the machining on the tip of the rubis is far superior and provides much better grip. " [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(1, device='cuda:3')


In [81]:
print(tokenizer.decode(errors[312][0]))
print("label: ", errors[312][1])
print("prediction", errors[312][2])

[CLS] it's amazing how much crud these things pull off of my nose. it was disgusting. imagine that constantly on ur nose! these are very necessary everyone should have them in their bathroom cabinet [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label:  tensor(4, device='cuda:3')
prediction tensor(0, device='cuda: